In [1]:
import math

In [2]:
gamma_file = open('Gamma2_V5.txt', 'r')
gamma = gamma_file.readlines()[0]
GAMMA = list(map(int, list(gamma)))
gamma_file.close()

сomb_file = open('CombFunc_V5.txt', 'r')
comb_f = сomb_file.readlines()[0].replace('} x', '*x').replace('_{', '').replace('}', '').replace('+', '⊕')
сomb_file.close()

In [3]:
print('f = ', comb_f)

f =  x0*x2*x5 ⊕ x1*x2*x5 ⊕ x1*x5 ⊕ x2*x3*x5 ⊕ x2*x4*x5 ⊕ x2*x5 ⊕ x2 ⊕ x4*x5


In [4]:
comb_f = lambda x0, x1, x2, x3, x4, x5: (x0*x2*x5 + x1*x2*x5 + x1*x5 + x2*x3*x5 + x2*x4*x5 + x2*x5 + x2 + x4*x5)%2

In [5]:
def int_to_bin(num, l):
    num = bin(num)[2:]
    while len(num) < l:
        num = '0' + num
    return [int(n) for n in num]

In [6]:
def dot(x, y):
    x, y = int_to_bin(x, 6), int_to_bin(y, 6)
    return sum(x[i]*y[i] for i in range(6))%2

In [7]:
coefs = [sum((-1)**((comb_f(*int_to_bin(x, 6)) + dot(alpha, x))%2) for x in range(2**6)) for alpha in range(2**6)]
sum(c**2 for c in coefs) == 2**(2*6)

True

In [8]:
non_zero_coefs_dict = {i: coefs[i] for i in range(2**6) if coefs[i] != 0}
for alpha in non_zero_coefs_dict:
    print(f'α = {alpha},  W_f(α) = {non_zero_coefs_dict[alpha]}')

α = 8,  W_f(α) = 32
α = 9,  W_f(α) = 32
α = 18,  W_f(α) = 16
α = 19,  W_f(α) = -16
α = 26,  W_f(α) = 16
α = 27,  W_f(α) = -16
α = 36,  W_f(α) = 16
α = 37,  W_f(α) = -16
α = 44,  W_f(α) = -16
α = 45,  W_f(α) = 16


In [9]:
weigths = [sum(list(map(int, bin(i)[2:]))) for i in range(2**6)]
sorted_weigths, sorted_coefs = zip(*sorted(zip(weigths, coefs)))
for i in range(2**6):
    if sorted_coefs[i] != 0:
        print('cor(f) =', sorted_weigths[i] - 1)
        break

cor(f) = 0


In [10]:
j = 1
for alpha in non_zero_coefs_dict:
    c = 1 if non_zero_coefs_dict[alpha] < 0 else 0
    a = int_to_bin(alpha, 6)
    g = ''.join(f'x{i} ⊕ ' for i in range(6) if a[i] == 1)[:-2] + (f'⊕ {c}' if c == 1 else '')
    print(f'{j}) g =', g, ', Pr{f(x) = g(x)} =', (1 + (-1)**c*non_zero_coefs_dict[alpha]/2**6)/2)
    j+= 1

1) g = x2  , Pr{f(x) = g(x)} = 3/4
2) g = x2 ⊕ x5  , Pr{f(x) = g(x)} = 3/4
3) g = x1 ⊕ x4  , Pr{f(x) = g(x)} = 5/8
4) g = x1 ⊕ x4 ⊕ x5 ⊕ 1 , Pr{f(x) = g(x)} = 5/8
5) g = x1 ⊕ x2 ⊕ x4  , Pr{f(x) = g(x)} = 5/8
6) g = x1 ⊕ x2 ⊕ x4 ⊕ x5 ⊕ 1 , Pr{f(x) = g(x)} = 5/8
7) g = x0 ⊕ x3  , Pr{f(x) = g(x)} = 5/8
8) g = x0 ⊕ x3 ⊕ x5 ⊕ 1 , Pr{f(x) = g(x)} = 5/8
9) g = x0 ⊕ x2 ⊕ x3 ⊕ 1 , Pr{f(x) = g(x)} = 5/8
10) g = x0 ⊕ x2 ⊕ x3 ⊕ x5  , Pr{f(x) = g(x)} = 5/8


In [12]:
polynomials = [x^10 + x^3 + 1, x^10 + x^7 + 1, x^10 + x^4 + x^3 + x^1 + 1, 
               x^10 + x^8 + x^3 + x^2 + 1, x^10 + x^8 + x^4 + x^3 + 1, x^10 + x^8 + x^5 + x^1 + 1]
polynomials_int_ = [''.join(str(i) for i in poly.list()[:-1]) for poly in polynomials]
polynomials_int = [int(''.join(str(i) for i in poly.list()[:-1]), 2) for poly in polynomials]

In [14]:
%%cython

from libc.stdlib cimport malloc, free

cdef int LENGTH = 10

def gen_sequence(int poly, int init_state, int nbits):
    cdef int curr_state = init_state, s, i, j
    cdef int *res_array = <int *>malloc(sizeof(int)*nbits)
    try:
        for i in range(nbits):
            res_array[i] = (curr_state >> (LENGTH - 1))&1
            s = 0
            for j in range(LENGTH):
                s = s^((( curr_state & poly ) >> j)&1)
            curr_state = (curr_state << 1)|s
        return [x for x in res_array[: nbits]]
    finally:
        free(res_array)

In [15]:
def hamming_dist(list1, list2):
    return sum(abs(list1[i] - list2[i]) for i in range(len(list1)))

# 1

$$g_1\Longrightarrow g_2\Longrightarrow g_{6}\Longrightarrow f$$

In [11]:
g1 = lambda x2: x2 
g2 = lambda x2, x5: (x2 + x5)%2
g6 = lambda x1, x2, x4, x5: (x1 + x2 + x4 + x5 + 1)%2

T1 = int(8*0.5**(-2)*math.log(2**(10*1)/0.01))
T2 = int(8*0.5**(-2)*math.log(2**(10*2)/0.01))
T4 = int(8*0.5**(-2)*math.log(2**(10*4)/0.01))

$$g_1=x_2$$

In [28]:
%%time
poss_states = {}
for i in range(2**10):
    seq = gen_sequence(polynomials_int[2], i, T1)
    poss_states[i] = hamming_dist(seq, GAMMA[:T1])
dist = min(poss_states.values())
print('min hamming distance = ', dist/T1)
poss_states = [i for i in poss_states if poss_states[i] == dist]
for s in poss_states:
    print('[X0, X1, X2, X3, X4, X5]: ', f'[-, -, {s}, -, -, -]') 

min hamming distance =  0.24390243902439024
[X0, X1, X2, X3, X4, X5]:  [-, -, 442, -, -, -]
CPU times: user 68.3 ms, sys: 1 ms, total: 69.3 ms
Wall time: 70.5 ms


$$g_2=x_2 ⊕ x_5$$

In [29]:
%%time
seq_x2 = gen_sequence(polynomials_int[2], 442, T2)
poss_states = {}
for i in range(2**10):
    seq_x5 = gen_sequence(polynomials_int[5], i, T2)
    seq = [g2(seq_x2[j], seq_x5[j]) for j in range(T2)]
    poss_states[i] = hamming_dist(seq, GAMMA[:T2])
dist = min(poss_states.values())
print('min hamming distance = ', (dist/T2).n())
poss_states = [i for i in poss_states if poss_states[i] == dist]
for s in poss_states:
    print('[X0, X1, X2, X3, X4, X5]: ', f'[-, -, 442, -, -, {s}]') 

min hamming distance =  0.245762711864407
[X0, X1, X2, X3, X4, X5]:  [-, -, 442, -, -, 997]
CPU times: user 816 ms, sys: 4.01 ms, total: 820 ms
Wall time: 819 ms


$$g_6=x_1⊕x_2⊕x_4⊕x_5⊕1$$

In [30]:
%%time
seq_x2 = gen_sequence(polynomials_int[2], 442, T4)
seq_x5 = gen_sequence(polynomials_int[5], 997, T4)
poss_states = {}
for i in range(2**10):
    seq_x1 = gen_sequence(polynomials_int[1], i, T4)
    for j in range(2**10):
        seq_x4 = gen_sequence(polynomials_int[4], j, T4)
        seq = [g6(seq_x1[k], seq_x2[k], seq_x4[k], seq_x5[k]) for k in range(T4)]
        poss_states[(i, j)] = hamming_dist(seq, GAMMA[:T4])
dist = min(poss_states.values())
print('min hamming distance = ', (dist/T4).n())
poss_states = [(i, j) for (i, j) in poss_states if poss_states[(i, j)] == dist]
for i, j in poss_states:
    print('[X0, X1, X2, X3, X4, X5]: ', f'[-, {i}, 442, -, {j}, 997]') 

min hamming distance =  0.400386847195358
[X0, X1, X2, X3, X4, X5]:  [-, 158, 442, -, 33, 997]
CPU times: user 28min 3s, sys: 360 ms, total: 28min 3s
Wall time: 28min 24s


$$f$$

In [31]:
%%time
seq_x1 = gen_sequence(polynomials_int[1], 158, 100)
seq_x2 = gen_sequence(polynomials_int[2], 442, 100)
seq_x4 = gen_sequence(polynomials_int[4], 33, 100)
seq_x5 = gen_sequence(polynomials_int[5], 997, 100)
triger = False
for i in range(2**10):
    if triger:
        break
    seq_x0 = gen_sequence(polynomials_int[0], i, 100)
    for j in range(2**10):
        seq_x3 = gen_sequence(polynomials_int[3], j, 100)
        seq = [comb_f(seq_x0[k], seq_x1[k], seq_x2[k], seq_x3[k], seq_x4[k], seq_x5[k]) for k in range(100)]
        if seq == GAMMA[:100]:
            print('[X0, X1, X2, X3, X4, X5]: ', f' [{i}, 158, 442, {j}, 33, 997]')
            triger = True
            break

[X0, X1, X2, X3, X4, X5]:   [830, 158, 442, 287, 33, 997]
CPU times: user 1min 52s, sys: 5 ms, total: 1min 52s
Wall time: 1min 53s


Перевірка для перших 1000 бітів:

In [24]:
seq_x0 = gen_sequence(polynomials_int[0], 830, 1000)
seq_x1 = gen_sequence(polynomials_int[1], 158, 1000)
seq_x2 = gen_sequence(polynomials_int[2], 442, 1000)
seq_x3 = gen_sequence(polynomials_int[3], 287, 1000)
seq_x4 = gen_sequence(polynomials_int[4], 33, 1000)
seq_x5 = gen_sequence(polynomials_int[5], 997, 1000)
seq = [comb_f(seq_x0[k], seq_x1[k], seq_x2[k], seq_x3[k], seq_x4[k], seq_x5[k]) for k in range(1000)]
seq == GAMMA[:1000]

True